In [1]:

#RandomForestClassifier
import math
import matplotlib as mpl
import warnings
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


excelFile = r'files/sw_train_data_sample.xlsx'
sample_df = pd.DataFrame(pd.read_excel(excelFile))
print(f"{sample_df.shape}")
sample_df.head()

(353, 7)


,code,name,kPattern,buy_price,sell_price,label_sell_price,label_buy_price
0,801012,农产品加工II,712,-1.098986,-0.008333,0.290301,-1.715114
1,801012,农产品加工II,712,-0.511574,0.225401,2.299587,-0.511574
2,801012,农产品加工II,712,2.521705,5.227228,8.238283,2.521705
3,801014,饲料II,712,-2.102709,-0.082342,3.585147,-2.102709
4,801014,饲料II,712,-0.882167,-0.200809,1.656360,-1.291246


In [2]:
def set_0_or_1(x):
    if x >= 2:
        return 1
    return 0

def percent_to_one(x):
    return int(x * 100) /1000.0

def to_one_min_max_120(x):
    v = x / 120
    if  v > 1:
        v = 1
    if  v< -1:
        v =-1
    return int(v*100)/100.0

def toInt(x):
    return int(x + 0.5)

d = sample_df
d['buy_price'] = d.buy_price.apply(percent_to_one)   #归一化
d['sell_price'] = d.sell_price.apply(percent_to_one)   #归一化
d['label_sell_price'] = d.label_sell_price.apply(toInt)
d['label_buy_price'] = d.label_buy_price.apply(toInt)
sample_df = sample_df.drop(columns=['code','kPattern', 'name'])
#sample_df.rename(columns={'d':'kd_diff'},inplace=True)
sample_df.head()

,buy_price,sell_price,label_sell_price,label_buy_price
0,-0.109,0.000,0,-1
1,-0.051,0.022,2,0
2,0.252,0.522,8,3
3,-0.210,-0.008,4,-1
4,-0.088,-0.020,2,0


In [3]:
data  = sample_df.values
x,y=np.split(data,indices_or_sections=(2,),axis=1) #x为数据，y为标签
y = y[:,0:1].astype('int')  #取第一列

In [4]:
import sklearn
from sklearn import model_selection
x_train,x_test,y_train,y_test =model_selection.train_test_split(x,y, train_size=0.7,test_size=0.3)

y_train = y_train.ravel()
y_test = y_test.ravel()
x_train.shape

(247, 2)

In [5]:
n_features = 20
#核心代码
#传统决策树、随机森林算法、极端随机树关于区别:https://blog.csdn.net/hanss2/article/details/53525503
#关于其中参数的说明请看http://www.jb51.net/article/131172.htm
clf1 = DecisionTreeClassifier(max_depth=None, min_samples_split=2,random_state=0)
clf2 = RandomForestClassifier(n_estimators=10,max_depth=None,min_samples_split=2, bootstrap=True)
clf3 = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2, bootstrap=False)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [6]:
clf2.fit(x_train,y_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [7]:
# 预测某个特征的在各个标签的概率
sampel_1 = x_test[5:10,]   ##取测试数据低第8,9个
sampel_1
predict_props = clf2.predict_proba(sampel_1)
predict_label = clf2.predict(sampel_1)
clf2.predict_log_proba(sampel_1)
predict_label,predict_props

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:704: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


(array([7, 2, 6, 0, 2]),
 array([[0. , 0. , 0. , 0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0.7, 0. , 0. ,
         0. , 0. ],
        [0. , 0. , 0. , 0. , 0.4, 0.5, 0.1, 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.1, 0.4, 0.4, 0.1, 0. ,
         0. , 0. ],
        [0. , 0. , 0. , 0.7, 0. , 0. , 0. , 0. , 0. , 0.3, 0. , 0. , 0. ,
         0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0.8, 0.2, 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. ]]))

In [9]:
label_total = np.unique(y_train)
label_total = np.sort(label_total)

predict_props.shape,label_total.shape,label_total

((5, 15),
 (15,),
 array([-3, -2, -1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 14]))

In [10]:

#交叉验证
scores1 = cross_val_score(clf1, x_train, y_train)
scores2 = cross_val_score(clf2, x_train, y_train)
scores3 = cross_val_score(clf3, x_train, y_train)
print()
print('DecisionTreeClassifier交叉验证准确率为:'+str(scores1.mean()))    
print('RandomForestClassifier交叉验证准确率为:'+str(scores2.mean()))    
print('ExtraTreesClassifier交叉验证准确率为:'+str(scores3.mean()))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)



DecisionTreeClassifier交叉验证准确率为:0.3198367346938776
RandomForestClassifier交叉验证准确率为:0.3318367346938775
ExtraTreesClassifier交叉验证准确率为:0.3158367346938776
